# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
import requests
import gmaps
import pprint
import seaborn as sb
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load city data
cities_file_path = "../output_data/cities.csv"

# Read the city data
weather_data_df = pd.read_csv(cities_file_path)

# Display the data table for preview
weather_data_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,Ixtapa,40,MX,1595303461,79,20.70,-105.20,84.20,3.36
1,1,São Filipe,41,CV,1595303461,82,14.90,-24.50,75.70,9.84
2,2,Hilo,90,US,1595303068,65,19.73,-155.09,82.40,8.05
3,3,George Town,20,MY,1595303199,79,5.41,100.34,87.80,4.70
4,4,Astoria,1,US,1595303402,82,46.19,-123.83,62.60,10.29
...,...,...,...,...,...,...,...,...,...,...
564,564,Chengde,100,CN,1595303414,39,40.97,117.94,89.74,1.30
565,565,Jieshi,96,CN,1595303531,65,22.81,115.83,86.74,6.06
566,566,Cravo Norte,100,CO,1595303531,94,6.30,-70.20,72.82,2.51
567,567,Praia da Vitória,75,PT,1595303531,73,38.73,-27.07,69.80,3.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

# Drop NaN values
weather_data_df = weather_data_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Lat", "Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity_rate = weather_data_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create cities dataframe
cities_weather_df = weather_data_df[['City', 'Cloudiness', 'Country', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Windspeed']]

#  Get the indices of cities that max temp between 70 & 80
cities_weather_df = cities_weather_df[(cities_weather_df['Max Temp']>70) & (cities_weather_df['Max Temp']<80)]

#  Get the indices of cities that have windspeed under 10 mph
cities_weather_df = cities_weather_df[(cities_weather_df['Windspeed']<10)]

#  Get the indices of cities that have windspeed under 10 mph
cities_weather_df = cities_weather_df[(cities_weather_df['Cloudiness'] == 0)]
cities_weather_df.dropna(inplace=True)
cities_weather_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Windspeed
121,San Patricio,0,MX,82,19.22,-104.70,79.48,8.37
262,Alta Floresta,0,BR,54,-9.88,-56.09,70.03,5.03
270,Qiryat Yam,0,IL,91,32.85,35.07,75.99,1.01
283,Cesena,0,IT,88,44.14,12.24,73.00,2.24
516,Zalţan,0,LY,86,32.95,11.87,70.18,5.39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = cities_weather_df.copy()
hotel_df['Hotel Name'] = ""

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel from df
    lat = row['Lat']
    lng = row['Lng']
    
    # geocoordinates
    target_coordinates = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)

        # convert response to json
        places_data = response.json()

        # Print the json (pretty printed)
        #print(json.dumps(places_data, indent=4, sort_keys=True))

        # Print the name and address of the first restaurant that appears
        #print(places_data["results"][0]["name"])
        #print(places_data["results"][0]["vicinity"])
        hotel_df[index] = places_data["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Windspeed,Hotel Name,121,262,283
121,San Patricio,0,MX,82,19.22,-104.70,79.48,8.37,,Grand Isla Navidad Resort,Lisboa Palace Hotel,Hotel Casali Home
262,Alta Floresta,0,BR,54,-9.88,-56.09,70.03,5.03,,Grand Isla Navidad Resort,Lisboa Palace Hotel,Hotel Casali Home
270,Qiryat Yam,0,IL,91,32.85,35.07,75.99,1.01,,Grand Isla Navidad Resort,Lisboa Palace Hotel,Hotel Casali Home
283,Cesena,0,IT,88,44.14,12.24,73.00,2.24,,Grand Isla Navidad Resort,Lisboa Palace Hotel,Hotel Casali Home
516,Zalţan,0,LY,86,32.95,11.87,70.18,5.39,,Grand Isla Navidad Resort,Lisboa Palace Hotel,Hotel Casali Home


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
